# NB10: LLM App Evaluation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RJuro/unistra-nlp2026/blob/main/notebooks/NB10_llm_evaluation.ipynb)

**Time:** ~40 min

## Learning Goals

- Design evaluation rubrics for LLM-based applications
- Implement automated metrics (accuracy, F1, precision, recall)
- Combine quantitative and qualitative evaluation approaches
- Understand RAGAS concepts for evaluating retrieval-augmented generation

In [ ]:
!pip install openai pandas scikit-learn numpy datasets -q

import pandas as pd
import numpy as np
import json
import time
from openai import OpenAI
from datasets import load_dataset
from sklearn.metrics import accuracy_score, classification_report, f1_score

## 1. Why Evaluation Matters

LLM outputs often *look* good -- they are fluent, coherent, and confident. But looking good is not the same as being correct. Without systematic evaluation, you can easily deploy a system that:

- Misclassifies edge cases that seem reasonable on the surface
- Hallucinates facts in a retrieval-augmented pipeline
- Degrades silently when the input distribution shifts

Evaluation is how we move from "it seems to work" to "we know it works, and we know where it fails."

There are two complementary approaches:

1. **Automated metrics** -- fast, reproducible, scalable. Use them to catch regressions and compare models.
2. **Human judgment** -- slower but catches things metrics miss. Use rubric-based scoring and qualitative error analysis.

A robust evaluation strategy uses **both**.

In [ ]:
GROQ_API_KEY = ""  # @param {type:"string"}

client = OpenAI(
    api_key=GROQ_API_KEY,
    base_url="https://api.groq.com/openai/v1"
)

## 2. Building an Eval Set

The foundation of any evaluation is a **gold-standard eval set** -- a curated collection of examples where we know the correct answer. This set should:

- Be representative of your real data distribution
- Include edge cases and difficult examples
- Have verified, high-quality ground-truth labels
- Be large enough to give stable metrics (30+ examples minimum, 100+ for per-class metrics)

We use the **tweet_eval stance detection** dataset (climate change topic). Stance classification asks: does this tweet express a **favorable**, **against**, or **neutral** position toward climate change action? This is inherently harder than topic classification — it requires understanding the author's *position*, not just the *subject*. This makes it an excellent testbed for evaluation.

In [ ]:
# Load tweet_eval stance dataset (climate change topic)
ds = load_dataset("cardiffnlp/tweet_eval", "stance_climate")

STANCE_LABELS = {0: "none", 1: "against", 2: "favor"}

# Convert test split to DataFrame
test_full = pd.DataFrame(ds["test"])
test_full["label_name"] = test_full["label"].map(STANCE_LABELS)

# Sample 30 examples as eval set
eval_set = test_full.sample(30, random_state=42)[['text', 'label_name']].reset_index(drop=True)
eval_set.columns = ['input_text', 'expected_label']

print(f"Eval set: {len(eval_set)} examples")
print(f"\nStance distribution:")
print(eval_set['expected_label'].value_counts())
print(f"\nExample tweet:")
print(eval_set.iloc[0]['input_text'][:200])

## 3. Automated Metrics

For classification tasks, the standard automated metrics are:

- **Accuracy** -- fraction of correct predictions. Simple but can be misleading with imbalanced classes.
- **Macro F1** -- average of per-class F1 scores. Treats all classes equally regardless of their size.
- **Per-class precision/recall** -- shows exactly where the model succeeds and where it fails.

Stance classification is particularly interesting because the three classes have **different confusion costs**: confusing "favor" with "against" is a much more serious error than confusing either with "none." We will run the LLM on every example in our eval set and compute these metrics.

In [ ]:
from openai import OpenAI
import json, time
from sklearn.metrics import accuracy_score, classification_report, f1_score

client = OpenAI(api_key=GROQ_API_KEY, base_url="https://api.groq.com/openai/v1")
CATEGORIES = ["none", "against", "favor"]

predictions = []
for _, row in eval_set.iterrows():
    try:
        resp = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[
                {"role": "system", "content": f"Determine the stance of this tweet toward climate change action. Classify into one of: {CATEGORIES}. 'favor' means supporting action on climate change, 'against' means opposing it, 'none' means neutral or unrelated. Return JSON with 'label' field only."},
                {"role": "user", "content": row['input_text'][:500]}
            ],
            response_format={"type": "json_object"},
            temperature=0.0,
            max_tokens=50
        )
        pred = json.loads(resp.choices[0].message.content).get('label', 'Unknown')
        predictions.append(pred)
    except:
        predictions.append('Unknown')
    time.sleep(0.1)

eval_set['predicted_label'] = predictions
valid = eval_set[eval_set.predicted_label != 'Unknown']

print(f"Accuracy: {accuracy_score(valid.expected_label, valid.predicted_label):.1%}")
print(f"Macro F1: {f1_score(valid.expected_label, valid.predicted_label, average='macro', zero_division=0):.3f}")
print(f"\n{classification_report(valid.expected_label, valid.predicted_label, zero_division=0)}")

## 4. Rubric-based Evaluation

Automated metrics give you a number, but they treat all errors equally. A rubric-based evaluation lets you define **degrees of correctness**. For stance classification, we define an asymmetric rubric:

- Confusing "favor" with "against" (or vice versa) is the **worst** error — the model got the position exactly backwards
- Confusing either stance with "none" is a **moderate** error — the model missed the stance but didn't invert it
- Exact match is a **perfect** score

We use an LLM as a **judge** to apply this rubric consistently. This is sometimes called "LLM-as-judge" evaluation.

In [ ]:
RUBRIC = """Score the stance classification on a scale of 1-5:
5: Exact match with ground truth
4: Close — predicted 'none' when true stance was mild, or vice versa
3: Partially correct — got the general sentiment but wrong specific label
2: Opposite direction — confused 'favor' with 'against' or vice versa
1: Completely wrong, no reasonable connection

Return JSON: {"score": <int>, "explanation": "<brief reason>"}"""

def score_with_rubric(input_text, expected, predicted):
    resp = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": RUBRIC},
            {"role": "user", "content": f"Tweet: {input_text[:200]}\nExpected stance: {expected}\nPredicted stance: {predicted}"}
        ],
        response_format={"type": "json_object"},
        temperature=0.0,
        max_tokens=100
    )
    return json.loads(resp.choices[0].message.content)

# Score a sample
scores = []
for _, row in eval_set.head(10).iterrows():
    s = score_with_rubric(row['input_text'], row['expected_label'], row['predicted_label'])
    scores.append(s)
    print(f"Score: {s.get('score','?')} | Expected: {row['expected_label']:<10} | Predicted: {row['predicted_label']:<10}")

avg_score = np.mean([s.get('score', 0) for s in scores if isinstance(s.get('score'), (int, float))])
print(f"\nAverage rubric score: {avg_score:.1f}/5")

## 5. Qualitative Error Analysis

This is often the most useful part of evaluation. Numbers tell you *how much* the model fails; error analysis tells you *why*.

Stance detection is particularly revealing for error analysis because:
- Some tweets use **sarcasm** — the model may take them literally
- Some tweets discuss climate change without taking a stance — the boundary between "none" and a mild stance is genuinely ambiguous
- Short tweets may lack sufficient context for reliable classification

Look at the actual misclassifications below. Are there patterns?

In [ ]:
errors = eval_set[eval_set.expected_label != eval_set.predicted_label]
print(f"Errors: {len(errors)}/{len(eval_set)} ({len(errors)/len(eval_set):.0%})")

# Check for the worst kind of error: favor↔against confusion
if len(errors) > 0:
    flipped = errors[
        ((errors.expected_label == 'favor') & (errors.predicted_label == 'against')) |
        ((errors.expected_label == 'against') & (errors.predicted_label == 'favor'))
    ]
    print(f"Stance inversions (favor↔against): {len(flipped)}/{len(errors)} errors")

for i, (_, row) in enumerate(errors.head(5).iterrows()):
    print(f"\n--- Error {i+1} ---")
    print(f"Tweet: {row['input_text'][:150]}...")
    print(f"Expected: {row['expected_label']}")
    print(f"Predicted: {row['predicted_label']}")

## 6. RAGAS Concepts for Retrieval

When evaluating **retrieval-augmented generation (RAG)** systems, classification metrics are not enough. You need to evaluate the full pipeline: retrieval quality *and* generation quality.

The [RAGAS](https://docs.ragas.io/) framework defines three key metrics:

1. **Faithfulness** -- Is the generated answer grounded in (supported by) the retrieved documents? A faithful answer does not add information beyond what the context provides.

2. **Context Relevance** -- Are the retrieved documents actually relevant to the question? Irrelevant context can mislead the generator.

3. **Answer Correctness** -- Is the final answer factually correct? This combines faithfulness with factual accuracy.

Stance classification connects to these ideas: when an LLM classifies stance, we can ask whether its *reasoning* is faithful to the tweet's actual content, or whether it is projecting assumptions. Below is a simple faithfulness check you can adapt for RAG evaluation.

In [ ]:
def check_faithfulness(question, context, answer):
    """Simple faithfulness check: is the answer supported by the context?"""
    resp = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": f"""Given this context and answer, is the answer faithful to (supported by) the context?
Context: {context[:500]}
Answer: {answer}
Return JSON: {{"faithful": true/false, "explanation": "brief reason"}}"""}],
        response_format={"type": "json_object"},
        temperature=0.0,
        max_tokens=100
    )
    return json.loads(resp.choices[0].message.content)

# Example
result = check_faithfulness(
    "What causes climate change?",
    "Climate change is primarily caused by greenhouse gas emissions from burning fossil fuels.",
    "Climate change is caused by solar activity."
)
print(f"Faithful: {result.get('faithful')} — {result.get('explanation')}")

## 7. Summary

A complete evaluation framework combines three approaches:

1. **Automated metrics** (accuracy, F1, precision/recall) -- fast, reproducible, good for regression testing and model comparison.
2. **Rubric-based scoring** (LLM-as-judge with defined criteria) -- captures degrees of correctness. For stance detection, this reveals that not all errors are equal.
3. **Qualitative error analysis** (manual inspection of failures) -- reveals *why* the system fails. For stance detection, common failure modes include sarcasm, implicit stance, and topic-adjacent tweets.

**Always use all three.** Metrics alone miss nuance. Rubrics alone miss scale. Qualitative analysis alone misses the big picture.

For RAG systems, add faithfulness and relevance checks (RAGAS concepts) to ensure the retrieval and generation components work together correctly.